In [1]:
from roboflow import Roboflow
import cv2
import numpy as np
import supervision as sv

In [28]:
cd C:\my_project\yolo_football_video_MoT

C:\my_project\yolo_football_video_MoT


In [39]:
rf = Roboflow(api_key="EQazb1etxBV71iH6TAIX")
project = rf.workspace('kshi2').project("chelsea_newcastle_test")
model = project.version(1312).model

loading Roboflow workspace...
loading Roboflow project...


In [26]:
mp4='C:/my_project/yolo_football_video_MoT/data/chelsea_newcastle_test.mp4'

In [29]:
cd .\prediction\video

C:\my_project\yolo_football_video_MoT\prediction\video


In [30]:
video_info=sv.VideoInfo.from_video_path(mp4)

In [31]:
video_info

VideoInfo(width=1920, height=1080, fps=30, total_frames=356)

In [40]:
def process_frame(frame: np.ndarray, _) -> np.ndarray:
    results = model(frame)[0]
    
    detections = sv.Detections.from_ultralytics(results)

    box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)

    labels = [f"{model.names[class_id]} {confidence:0.2f}" for _, _, confidence, class_id, _ in detections]
    frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)

    return frame

In [41]:
sv.process_video(source_path=mp4, target_path="result.mp4", callback=process_frame)

TypeError: 'ObjectDetectionModel' object is not callable

In [6]:
job_id, signed_url, expire_time = model.predict_video(
    mp4,
    fps=5,
    prediction_type="batch-video",
)

results = model.poll_until_video_results(job_id)

print(results)

Checking for video inference results for job bc9daf41-cc0e-4973-8067-5a554c36871e every 60s
(0s): Checking for inference results
{'frame_offset': [0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96, 102, 108, 114, 120, 126, 132, 138, 144, 150, 156, 162, 168, 174, 180, 186, 192, 198, 204, 210, 216, 222, 228, 234, 240, 246, 252, 258, 264, 270, 276, 282, 288, 294, 300, 306, 312, 318, 324, 330, 336, 342, 348], 'time_offset': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6, 2.8, 3.0, 3.2, 3.4, 3.6, 3.8, 4.0, 4.2, 4.4, 4.6, 4.8, 5.0, 5.2, 5.4, 5.6, 5.8, 6.0, 6.2, 6.4, 6.6, 6.8, 7.0, 7.2, 7.4, 7.6, 7.8, 8.0, 8.2, 8.4, 8.6, 8.8, 9.0, 9.2, 9.4, 9.6, 9.8, 10.0, 10.2, 10.4, 10.6, 10.8, 11.0, 11.2, 11.4, 11.6], 'chelsea_newcastle_test': [{'time': 0.17586495261639357, 'image': {'width': 1920, 'height': 1080}, 'predictions': [{'x': 779.0, 'y': 471.0, 'width': 72.0, 'height': 114.0, 'confidence': 0.9598721861839294, 'class': 'Chelsea', 'class_id': 1, 'detection_id'

In [7]:
results.keys()

dict_keys(['frame_offset', 'time_offset', 'chelsea_newcastle_test'])

In [10]:
results['chelsea_newcastle_test']

[{'time': 0.17586495261639357,
  'image': {'width': 1920, 'height': 1080},
  'predictions': [{'x': 779.0,
    'y': 471.0,
    'width': 72.0,
    'height': 114.0,
    'confidence': 0.9598721861839294,
    'class': 'Chelsea',
    'class_id': 1,
    'detection_id': '6ff1791e-fedc-443b-af6c-75a168067e47'},
   {'x': 569.5,
    'y': 492.5,
    'width': 71.0,
    'height': 119.0,
    'confidence': 0.9411757588386536,
    'class': 'Newcastle',
    'class_id': 3,
    'detection_id': '911ad8fd-7bf9-49eb-89cc-272a1fac6721'},
   {'x': 973.0,
    'y': 403.0,
    'width': 62.0,
    'height': 88.0,
    'confidence': 0.9323234558105469,
    'class': 'Newcastle',
    'class_id': 3,
    'detection_id': 'fd37e896-c34f-4e90-970c-2bfc8eba4c55'},
   {'x': 1808.5,
    'y': 594.5,
    'width': 105.0,
    'height': 153.0,
    'confidence': 0.9292673468589783,
    'class': 'Chelsea',
    'class_id': 1,
    'detection_id': '6a947a5c-0d21-4f1c-b338-f6ea51e1d2ae'},
   {'x': 926.5,
    'y': 387.5,
    'width': 49.0

In [11]:
# 비디오 쓰기 객체 생성
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (640, 480))

In [12]:
def callback(scene: np.ndarray, index: int) -> np.ndarray:
    results = results[index]
    detections = sv.Detections.from_ultralytics(results)

    bounding_box_annotator = sv.BoundingBoxAnnotator()
    label_annotator = sv.LabelAnnotator()

    labels = [
        results.names[class_id]
        for class_id
        in detections.class_id
    ]

    annotated_image = bounding_box_annotator.annotate(
        scene=scene, detections=detections)

    annotated_image = label_annotator.annotate(
        scene=annotated_image, detections=detections, labels=labels)

    return annotated_image

sv.process_video(
    source_path=mp4,
    target_path="output.mp4",
    callback=callback
)

UnboundLocalError: cannot access local variable 'results' where it is not associated with a value

In [22]:
mp4read=cv2.VideoCapture(mp4)
cap,frame=mp4read.read()

In [29]:
frame[:,:,:].shape

(1080, 1920, 3)

In [7]:
# Hex 색상코드를 RGB색상코드로 변경
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

In [11]:
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        # 프레임에서 객체 감지
        prediction = model.predict(frame, confidence=50)
        
        # 결과를 프레임에 표시
        for result in prediction:
            bottom_right = (round(result['x']-result['width']/2), round(result['y']-result['height']/2))
            top_left = (round(result['x']+result['width']/2), round(result['y']+result['height']/2))
            color = hex_to_rgb(version.colors[result['class']])
            thickness = 2
            cv2.rectangle(frame, bottom_right, top_left, color, thickness)
            cv2.putText(frame, result['class'], (round(result['x']-result['width']/2), round(result['y']-result['height']/2-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color=color, thickness=3)
        
        # 결과 프레임을 파일로 저장
        out.write(frame)
    else:
        break

# 모든 작업이 완료되면 캡처와 쓰기 객체를 해제
cap.release()
out.release()

HTTPError: 403 Client Error: Forbidden for url: https://detect.roboflow.com/chelsea_newcastle/1?api_key=7LaCX25FUT6blBXw40FE&name=YOUR_IMAGE.jpg&overlap=30&confidence=50&stroke=1&labels=false&format=json